В папке subsid (shared/homeworks/python_ds_miniprojects/5_subsid) находятся файлы (tm_sales_1, tm_sales_2, ...) с продажами продуктов через телемаркетинг. Каждый файл содержит, как минимум, 4 колонки (поля): FILIAL_ID, SUBS_ID, PROD_ID, ACT_DTTM.

Суть задачи в том, чтобы проверить подключения продуктов определенным пользователям, соединив файлы о продажах с логами по подключениям в системе.

Особенности данных:
сотрудники телемаркетинга не всегда указывают полный id, если 'id' нет в начале SUBS_ID, то нужно его добавить(+)
поля в файлах могут быть расположены абсолютно случайным образом, но названия полей статичны
продажа не засчитывается, если отключение (END_DTTM) произошло меньше чем через 5 минут после подключения (START_DTTM)
если в файле с продажами встречается строка без указанного SUBS_ID, она пропускается(+)
Сохраните результат в датафрэйм с разделителем ;, содержащий корректные подключения.

Note: обратите внимание на то, как pandas переводит дату из строки, возможно вам понадобится параметр format
Пример содержания итогового файла (колонка difference это разница между START_DTTM и END_DTTM):

In [15]:
import pandas as pd
import os

In [95]:
df_1 = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-p-rudenko/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/tm_sales_1.csv', sep = ';')
df_2 = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-p-rudenko/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/tm_sales_2.csv', sep = ';')
df_3 = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-p-rudenko/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/tm_sales_3.csv', sep = ';')
df_4 = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-p-rudenko/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/prod_activations_logs.csv', sep = ';')

In [14]:
#path = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-p-rudenko/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/'

In [173]:
#соединение датафреймов
tsm_sales = pd.DataFrame()
tsm_sales = pd.concat([df_1,df_2])
tsm_sales = pd.concat([tsm_sales,df_3])

In [175]:
#убираем строки, по которым нет значений в колонке SUBS_ID
tsm_sales = tsm_sales.dropna(subset=['SUBS_ID'])

In [176]:
#ресетим индекс и убираем колонку index
tsm_sales = tsm_sales.reset_index().drop(columns = 'index')

In [296]:
tsm_sales

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,id7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id5416547,3,1743,17-03-2020 10:17
4,id8362218,7,9879,05-03-2020 11:42
5,id2185490,2,3210,16-03-2020 16:28
6,id5764122,3,1499,18-03-2020 15:44
7,id7642700,6,3020,15-03-2020 14:21
8,id1374509,2,5677,17-03-2020 11:48


In [177]:
# функция, добавляющая id в начало строки, если строка начинается не с id
def sign (x):
    if not str(x).startswith('id'):
        x = str('id' + x)
    return x

In [179]:
#применяем функцию к колонке SUBS_ID
tsm_sales['SUBS_ID'] = tsm_sales['SUBS_ID'].apply(sign)

In [246]:
#создаем пустой датафрейм
df = pd.DataFrame()

In [304]:
# (иннер)джойним две таблицы по айди 
df = tsm_sales.merge (df_4, on = ['SUBS_ID','PROD_ID'])

In [309]:
#переводим в формат даты с указанием формата format='%d-%m-%Y %H:%M' для российской действительности
df['START_DTTM'] = pd.to_datetime(df['START_DTTM'], format='%d-%m-%Y %H:%M')
df['END_DTTM'] = pd.to_datetime(df['END_DTTM'], format='%d-%m-%Y %H:%M')

In [311]:
#считаем разницу между концом и началом звонка
df['difference'] = (df.END_DTTM - df.START_DTTM)

In [315]:
#по условию задачи берем только те значения, где разница больше 5 минут
df[df.difference > '5m'].SUBS_ID.sort_values().values

array(['id2185490', 'id4651830', 'id5416547', 'id7642700'], dtype=object)